In [1]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, split, concat, col, lit
from pyspark.sql.types import StructType, StructField, LongType, StringType, DoubleType
from time import sleep

In [2]:
spark_conf = SparkConf()
spark_conf.setMaster("spark://master:7077")
spark_conf.setAppName("Lab7_9")
spark_conf.set("spark.driver.memory", "2g")
spark_conf.set("spark.executor.cores", "1")
spark_conf.set("spark.driver.cores", "1")

# create the spark session, which is the entry point to Spark SQL engine.
spark = SparkSession.builder.config(conf=spark_conf).getOrCreate()

In [3]:
# Read the whole dataset as a batch
df = spark.readStream.format("kafka").option("kafka.bootstrap.servers", "kafka:9093").option("subscribe", "word").option("startingOffsets", "earliest").load()

In [4]:
lines = df.selectExpr("CAST(value AS STRING)")

words = lines.select(explode(split(lines.value, " ")).alias("word"))

word_counts = words.groupBy("word").count()

In [ ]:
query = word_counts.select(concat(col("word"), lit(" "), col("count")).alias("value")).writeStream.format("kafka").option("kafka.bootstrap.servers", "kafka:9093").option("checkpointLocation", "../checkpoint").option("topic", "wordcount").outputMode("complete").start()

query.awaitTermination()

In [6]:
# Stop the spark context
spark.stop()